## Import

In [ ]:
import beamtest_analysis_helper as helper
from glob import glob
from natsort import natsorted
import pandas as pd
import hist
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.colors as colors
import mplhep as hep
hep.style.use('CMS')

In [ ]:
chip_labels = [0, 1, 2, 3]
chip_names = ["ET2_EPIR_Pair1", "ET2_BAR_4", "ET2_BAR_6", "ET2_EPIR_Pair4"]
offsets = [20, 20, 20, 20]
high_voltages = [250, 250, 250, 250]

fig_path = './'

board_to_analyze = [0, 1, 3]

chip_fignames = chip_names
chip_figtitles = [
    f"(Trigger) Pair1 HV{high_voltages[0]}V OS:{offsets[0]}",
    f"Bar4 HV{high_voltages[1]}V OS:{offsets[1]}",
    f"Bar6 HV{high_voltages[2]}V OS:{offsets[2]}",
    f"Pair4 HV{high_voltages[3]}V OS:{offsets[3]}"]

## Loading dataframe: No trigger, No event

In [ ]:
files = glob('./DESYFeb2024_Run_0_loop_10.feather')

last_evt = 0
dataframes = []

for idx, ifile in enumerate(files):
    tmp_df = pd.read_feather(ifile)
    tmp_df.drop(columns=['evt_number', 'bcid', 'l1a_counter', 'ea'], inplace=True)

    if idx > 0:
        tmp_df['evt'] += last_evt
    last_evt += tmp_df['evt'].unique()[-1]

    event_board_counts = tmp_df.groupby(['evt', 'board']).size().unstack(fill_value=0)

    base_selection = (event_board_counts[0] >= 1) # event must have at least one hit from the trigger board
    selected_subset_df = tmp_df[tmp_df['evt'].isin(event_board_counts[base_selection].index)]
    selected_subset_df.reset_index(inplace=True, drop=True)

    dataframes.append(selected_subset_df)
    del tmp_df, event_board_counts, base_selection, selected_subset_df

df = pd.concat(dataframes)
df.reset_index(inplace=True, drop=True)
del dataframes

In [ ]:
df.info()

## Inclusive plots

In [ ]:
helper.plot_number_of_fired_board(df)

In [ ]:
helper.plot_number_of_hits_per_event(df, fig_titles=chip_figtitles)

In [ ]:
helper.plot_2d_nHits_nBoard(df, fig_titles=chip_figtitles)

In [ ]:
h_inclusive = helper.return_hist(df, chip_names, chip_labels, hist_bins=[100, 128, 128])

for iboard in board_to_analyze:
    helper.plot_1d_TDC_histograms(h_inclusive, chip_names[iboard], chip_fignames[iboard], chip_figtitles[iboard], fig_path, save=False,
                                tag="inclusive", fig_tag=", inclusive", slide_friendly=True)

del h_inclusive

In [ ]:
helper.plot_occupany_map(df, chipLabels=chip_labels, chipNames=chip_names, fig_title=chip_figtitles, fig_tag='inclusive')

## Filter noise

In [ ]:
## Selecting good hits
tdc_cuts = {}
for idx in board_to_analyze:
    # board ID: [CAL LB, CAL UB, TOA LB, TOA UB, TOT LB, TOT UB]
    if idx == 0:
        tdc_cuts[idx] = [df.loc[df['board'] == idx]['cal'].mode()[0]-50, df.loc[df['board'] == idx]['cal'].mode()[0]+50,  100, 500, 0, 600]
    else:
        tdc_cuts[idx] = [df.loc[df['board'] == idx]['cal'].mode()[0]-50, df.loc[df['board'] == idx]['cal'].mode()[0]+50,  0, 1100, 0, 600]

filtered_df = helper.tdc_event_selection(df, tdc_cuts_dict=tdc_cuts, select_by_hit=False)
del df

In [ ]:
filtered_df

## Correlation trigger - reference

In [ ]:
event_board_counts = filtered_df.groupby(['evt', 'board']).size().unstack(fill_value=0)
event_selection_col = None

trig_selection = (event_board_counts[0] == 1)
ref_selection = (event_board_counts[3] == 1)
event_selection_col = trig_selection & ref_selection

selected_event_numbers = event_board_counts[event_selection_col].index
selected_subset_df = filtered_df[filtered_df['evt'].isin(selected_event_numbers)]
selected_subset_df.reset_index(inplace=True, drop=True)

single_df = selected_subset_df
del selected_subset_df

In [ ]:
event_board_counts = filtered_df.groupby(['evt', 'board']).size().unstack(fill_value=0)
event_selection_col = None

trig_selection = (event_board_counts[0] == 1)
ref_selection = (event_board_counts[3] > 1)
event_selection_col = trig_selection & ref_selection

selected_event_numbers = event_board_counts[event_selection_col].index
selected_subset_df = filtered_df[filtered_df['evt'].isin(selected_event_numbers)]
selected_subset_df.reset_index(inplace=True, drop=True)

multiple_df = selected_subset_df
del selected_subset_df

In [ ]:
broadcasted_multiple_df = multiple_df.groupby('evt').apply(helper.broadcast_dataframe, reference_board_id=3, board_id_want_broadcast=0).reset_index(drop=True)

In [ ]:
broadcasted_oneTrig_onePlusRef_df = pd.concat([single_df.loc[(single_df['board'] == 0) | (single_df['board'] == 3)].reset_index(drop=True), broadcasted_multiple_df])
del multiple_df

In [ ]:
helper.plot_correlation_of_pixels(single_df, np.array([0,3]), xaxis_label_board_name='Reference Board', fig_title="Single hit on Trig and Ref", fit_tag='')

In [ ]:
helper.plot_correlation_of_pixels(broadcasted_multiple_df, np.array([0,3]), xaxis_label_board_name='Reference Board', fig_title="Single hit on Trig / Multi hits on Ref", fit_tag='')

In [ ]:
h_dis_1hit = helper.plot_distance(single_df, board_ids=np.array([0, 3]), xaxis_label_board_name='Reference', fig_title="Single hit on Trig and Ref", do_logy=False)

In [ ]:
h_dis_multihits = helper.plot_distance(broadcasted_multiple_df, board_ids=np.array([0, 3]), xaxis_label_board_name='Reference', fig_title="Single hit on Trig / Multi hits on Ref", do_logy=False)

## Compare Ref 1 hit and 2+ hit cases

In [ ]:
fig, ax = plt.subplots(dpi=100, figsize=(15, 8))
hep.histplot(h_dis_1hit, density=True, label="1= Ref Hit")
hep.histplot(h_dis_multihits, density=True, label="2+ Ref Hits")
hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
ax.legend()

## Trigger - DUT Correlations (Only 1 Hit on Trig + 1 Hit on Ref + 1 Hit on DUT)

In [ ]:
event_board_counts = oneTrig_onePlusRef_df.groupby(['evt', 'board']).size().unstack(fill_value=0)

ref_selection = (event_board_counts[2] == 1)
dut_selection = (event_board_counts[1] == 1)
event_selection_col = ref_selection & dut_selection

selected_event_numbers = event_board_counts[event_selection_col].index
tmp_df = oneTrig_onePlusRef_df[oneTrig_onePlusRef_df['evt'].isin(selected_event_numbers)]
tmp_df.reset_index(inplace=True, drop=True)

In [ ]:
helper.plot_correlation_of_pixels(tmp_df, hit_type='single', board_id_to_correlate=1, title_tag='')
h_dis_dut_one = helper.plot_distance(tmp_df, hit_type='single', board_id_to_correlate=1, title_tag='', do_logy=True)

In [ ]:
del tmp_df, event_board_counts, selected_event_numbers

## Trigger - DUT Correlations (Only 1 Hit on Trig + 1 Hit on Ref + 2+ Hits on DUT)

In [ ]:
event_board_counts = oneTrig_onePlusRef_df.groupby(['evt', 'board']).size().unstack(fill_value=0)

ref_selection = (event_board_counts[2] == 1)
dut_selection = (event_board_counts[1] >= 2)
event_selection_col = ref_selection & dut_selection

selected_event_numbers = event_board_counts[event_selection_col].index
tmp_df = oneTrig_onePlusRef_df[oneTrig_onePlusRef_df['evt'].isin(selected_event_numbers)]
tmp_df.reset_index(inplace=True, drop=True)

In [ ]:
helper.plot_correlation_of_pixels(tmp_df, hit_type='multiple', board_id_to_correlate=1, title_tag='')
h_dis_dut_2p = helper.plot_distance(tmp_df, hit_type='multiple', board_id_to_correlate=1, title_tag='', do_logy=True)

In [ ]:
del tmp_df, event_board_counts, selected_event_numbers

## Compare different DUT cases

In [ ]:
fig, ax = plt.subplots(dpi=100, figsize=(15, 8))
hep.histplot(h_dis_dut_one, density=True, label="=1 DUT Hits")
hep.histplot(h_dis_dut_2p, density=True, label="2+ DUT Hits")
hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
# ax.set_title(f"{title_tag}", loc="right", size=15)
ax.legend()